In [ ]:
ptbr 0.626~0.71~0.740
ptpt 0.706~0.73~0.784

Outras similaridades: Cos, Ovl, Jacc, Dice

# Imports e métodos necessários

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import pandas as pd 
import xml.etree.ElementTree as et 

from scipy.stats import pearsonr

def parse_xml(xml_file):
    """Parse xml to pandas dataframe."""
    xtree = et.parse(xml_file)
    xroot = xtree.getroot() 

    df_cols = ['id', 't', 'h', 'similarity']
    rows = []

    for node in xroot:
        id_ = node.attrib.get("id")
        similarity = node.attrib.get("similarity")
        t = node.find("t").text
        h = node.find("h").text

        rows.append({
            "id": id_,
            "t": t, 
            "h": h,
            "similarity": similarity
        })
    return pd.DataFrame(rows, columns=df_cols, dtype=float)

def eval_similarity(pairs_gold, pairs_sys):
    '''
    Evaluate the semantic similarity output of the system against a gold score. 
    Results are printed to stdout.
    '''
    
    gold_values = np.array(pairs_gold)
    sys_values = np.array(pairs_sys)
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()
    
    print()
    print('Similarity evaluation')
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.3f}\t\t{:18.2f}'.format(pearson, mse))

# Carregando os dados

In [3]:
!ls ../data/assin

assin-ptbr-dev.xml   assin-ptbr-train.xml  assin-ptpt-test.xml
assin-ptbr-test.xml  assin-ptpt-dev.xml    assin-ptpt-train.xml


In [4]:
df_ptbr_train = parse_xml('../data/assin/assin-ptbr-train.xml')
df_ptbr_dev = parse_xml('../data/assin/assin-ptbr-dev.xml')
df_ptbr_test = parse_xml('../data/assin/assin-ptbr-test.xml')

df_ptpt_train = parse_xml('../data/assin/assin-ptpt-train.xml')
df_ptpt_dev = parse_xml('../data/assin/assin-ptpt-dev.xml')
df_ptpt_test = parse_xml('../data/assin/assin-ptpt-test.xml')

In [5]:
print(f'assin-ptbr-train: {df_ptbr_train.shape}')
print(f'assin-ptbr-dev: {df_ptbr_dev.shape}')
print(f'assin-ptbr-test: {df_ptbr_test.shape}')
print()
print(f'assin-ptpt-train: {df_ptpt_train.shape}')
print(f'assin-ptpt-dev: {df_ptpt_dev.shape}')
print(f'assin-ptpt-test: {df_ptpt_test.shape}')

assin-ptbr-train: (2500, 4)
assin-ptbr-dev: (500, 4)
assin-ptbr-test: (2000, 4)

assin-ptpt-train: (2500, 4)
assin-ptpt-dev: (500, 4)
assin-ptpt-test: (2000, 4)


In [6]:
df_ptbr_train.head()

,id,t,h,similarity
0,1.0,"A gente faz o aporte financeiro, é como se a e...",Fernando Moraes afirma que não tem vínculo com...,2.00
1,2.0,"Em 2013, a história de como Walt Disney conven...",P.L.Travers era completamente contra a adaptaç...,2.25
2,3.0,"David Silva bateu escanteio, Kompany escalou a...","David Silva cobrou escanteio, o zagueiro se ap...",3.75
3,4.0,"Para os ambientalistas, as metas anunciadas pe...","Dilma aproveitou seu discurso ontem, na Confer...",2.75
4,5.0,"De acordo com a PM, por volta das 10h30 havia ...",O protesto encerrou por volta de 12h15 (horári...,2.00


# Testes pt-br

## BERT

In [7]:
from typing import Callable, List, Optional, Tuple

import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
import torch


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.tokenizer = bert_tokenizer
        self.model = bert_model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(text,
                                                    add_special_tokens=True,
                                                    max_length=self.max_length
                                                    )["input_ids"]

        # Create an attention mask telling BERT to use all words
        attention_mask = [1] * len(tokenized_text)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(tokenized_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self

### bert-base-portuguse-cased

In [8]:
from transformers import BertModel, BertTokenizer

pretrained_model = '../data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
bert_model = BertModel.from_pretrained(pretrained_model)

bert_transformer = BertTransformer(tokenizer, bert_model)

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
t_embeddings = bert_transformer.transform(df_ptbr_test['t'].tolist())
h_embeddings = bert_transformer.transform(df_ptbr_test['h'].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings.numpy(), h_embeddings.numpy())]

In [11]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [12]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.444		              1.97


In [13]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.475


### bert-large-portuguese-cased

In [14]:
from transformers import BertModel, BertTokenizer

pretrained_model = '../data/PyTorch checkpoint/bert-large-portuguese-cased_pytorch_checkpoint'
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
bert_model = BertModel.from_pretrained(pretrained_model)

bert_transformer = BertTransformer(tokenizer, bert_model)

In [15]:
t_embeddings = bert_transformer.transform(df_ptbr_test['t'].tolist())
h_embeddings = bert_transformer.transform(df_ptbr_test['h'].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [16]:
similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings.numpy(), h_embeddings.numpy())]

In [17]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [18]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.554		              3.32


In [19]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.561


## Sentence-BERT

### bert-base-portuguese-cased

In [20]:
from sentence_transformers import models, SentenceTransformer
# Use BERT for mapping tokens to embeddings

pretrained_model = '../data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
word_embedding_model = models.Transformer(pretrained_model)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [21]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [22]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [23]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [24]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.573		              1.60


In [25]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.576


### bert-large-portuguese-cased

In [26]:
from sentence_transformers import models, SentenceTransformer
# Use BERT for mapping tokens to embeddings

pretrained_model = '../data/PyTorch checkpoint/bert-large-portuguese-cased_pytorch_checkpoint'
word_embedding_model = models.Transformer(pretrained_model)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [27]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [28]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [29]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [30]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.582		              2.69


In [31]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.598


## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### bert-base-portuguese-cased

In [7]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(10):
    # Read the dataset
    model_name = '/mnt/data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_base_finetuning_assin-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)

    word_embedding_model = models.Transformer(model_name)

    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)
    
    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.769		              0.34
Spearman correlation:   0.760





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.763		              0.35
Spearman correlation:   0.756





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.764		              0.34
Spearman correlation:   0.759





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.767		              0.34
Spearman correlation:   0.761


...




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.763		              0.35
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.751		              0.38
Spearman correlation:   0.746





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.765		              0.35
Spearman correlation:   0.758





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.773		              0.33
Spearman correlation:   0.768





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.767		              0.34
Spearman correlation:   0.760


### bert-large-portuguese-cased

In [8]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(10):
    # Read the dataset
    model_name = '/mnt/data/PyTorch checkpoint/bert-large-portuguese-cased_pytorch_checkpoint'
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_finetuning_assin-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)

    word_embedding_model = models.Transformer(model_name)

    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.790		              0.33
Spearman correlation:   0.784





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.784		              0.33
Spearman correlation:   0.780





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.769		              0.34
Spearman correlation:   0.763





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.778		              0.34
Spearman correlation:   0.776





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.787		              0.33
Spearman correlation:   0.781





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.783		              0.35
Spearman correlation:   0.777





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.786		              0.33
Spearman correlation:   0.783





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.33
Spearman correlation:   0.788





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.787		              0.33
Spearman correlation:   0.780





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.788		              0.32
Spearman correlation:   0.782


# Testes pt-pt

## BERT

### bert-base-portuguese-cased

In [32]:
from transformers import BertModel, BertTokenizer

pretrained_model = '../data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
bert_model = BertModel.from_pretrained(pretrained_model)

bert_transformer = BertTransformer(tokenizer, bert_model)

In [33]:
t_embeddings = bert_transformer.transform(df_ptpt_test['t'].tolist())
h_embeddings = bert_transformer.transform(df_ptpt_test['h'].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [34]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings.numpy(), h_embeddings.numpy())]

In [35]:
pairs_gold = df_ptpt_test['similarity'].tolist()
pairs_sys = similarities

In [36]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.379		              2.46


In [37]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.408


### bert-large-portuguese-cased

In [8]:
from transformers import BertModel, BertTokenizer

pretrained_model = '../data/PyTorch checkpoint/bert-large-portuguese-cased_pytorch_checkpoint'
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
bert_model = BertModel.from_pretrained(pretrained_model)

bert_transformer = BertTransformer(tokenizer, bert_model)

In [9]:
t_embeddings = bert_transformer.transform(df_ptpt_test['t'].tolist())
h_embeddings = bert_transformer.transform(df_ptpt_test['h'].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [11]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings.numpy(), h_embeddings.numpy())]

In [15]:
pairs_gold = df_ptpt_test['similarity'].tolist()
pairs_sys = similarities

In [16]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.521		              4.16


In [17]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.530


## Sentence-BERT

### bert-base-portuguese-cased

In [44]:
from sentence_transformers import models, SentenceTransformer
# Use BERT for mapping tokens to embeddings

pretrained_model = '../data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
word_embedding_model = models.Transformer(pretrained_model)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [45]:
t_embeddings = model.encode(df_ptpt_test['t'].tolist())
h_embeddings = model.encode(df_ptpt_test['h'].tolist())

In [46]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [47]:
pairs_gold = df_ptpt_test['similarity'].tolist()
pairs_sys = similarities

In [48]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.531		              2.22


In [49]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.537


### bert-large-portuguese-cased

In [50]:
from sentence_transformers import models, SentenceTransformer
# Use BERT for mapping tokens to embeddings

pretrained_model = '../data/PyTorch checkpoint/bert-large-portuguese-cased_pytorch_checkpoint'
word_embedding_model = models.Transformer(pretrained_model)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [51]:
t_embeddings = model.encode(df_ptpt_test['t'].tolist())
h_embeddings = model.encode(df_ptpt_test['h'].tolist())

In [52]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [53]:
pairs_gold = df_ptpt_test['similarity'].tolist()
pairs_sys = similarities

In [54]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.491		              3.57


In [55]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.512


## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### bert-base-portuguese-cased

In [9]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(10):
    # Read the dataset
    model_name = '/mnt/data/PyTorch checkpoint/bert-base-portuguese-cased_pytorch_checkpoint'
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_base_ptpt_finetuning_assin-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)

    word_embedding_model = models.Transformer(model_name)

    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptpt_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptpt_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptpt_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.42
Spearman correlation:   0.723





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.742		              0.40
Spearman correlation:   0.733





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.41
Spearman correlation:   0.723





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.732		              0.41
Spearman correlation:   0.722





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.740		              0.41
Spearman correlation:   0.730





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.735		              0.40
Spearman correlation:   0.724





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.742		              0.40
Spearman correlation:   0.733





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.734		              0.41
Spearman correlation:   0.723





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.734		              0.42
Spearman correlation:   0.724





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.735		              0.42
Spearman correlation:   0.724


### bert-large-portuguse-cased

In [10]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(10):
    # Read the dataset
    model_name = '/mnt/data/PyTorch checkpoint/bert-large-portuguese-cased_pytorch_checkpoint'
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_ptpt_finetuning_assin-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)

    word_embedding_model = models.Transformer(model_name)

    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptpt_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptpt_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptpt_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.758		              0.41
Spearman correlation:   0.752





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.754		              0.40
Spearman correlation:   0.748





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.759		              0.42
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.747		              0.41
Spearman correlation:   0.738





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.764		              0.40
Spearman correlation:   0.755


...



Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.764		              0.35
Spearman correlation:   0.759





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.741		              0.43
Spearman correlation:   0.730





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.748		              0.42
Spearman correlation:   0.742





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.752		              0.40
Spearman correlation:   0.747





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.764		              0.40
Spearman correlation:   0.758
